# Targeting high value customers based on customer demographics and attributes.
> This project was done under the umbrella of KPMG internship experience. I was provided data sets of an organization targeting a client who wants a feedback from us on their dataset quality and how this can be improved.

### Background
- Sprocket Central Pty Ltd, a medium size bikes & cycling accessories organisation
- needs help with its customer and transactions data
- how to analyse it to help optimise its marketing strategy effectively.

### Datasets
- New Customer List
- Customer Demographic
- Customer Addresses
- Transactions data in the past 3 months


### Task
- Exploratory Data Analysis to understand the data and its quality
- Model building to predict the high value customers
- Results and recommendations

In [104]:
# Importing libraries

import pandas as pd
import numpy as np

In [105]:
# Importing data
xls = pd.ExcelFile('KPMG_VI_New_raw_data_update_final.xlsx')

Transactions = pd.read_excel(xls, 'Transactions', skiprows=1)
NewCustomerList = pd.read_excel(xls, 'NewCustomerList', skiprows=1)
Demographic = pd.read_excel(xls, 'CustomerDemographic', skiprows=1)
Address = pd.read_excel(xls, 'CustomerAddress', skiprows=1)

### Checking correlation and common columns among the sheets

In [106]:
transactions_columns = Transactions.columns
demographic_columns = Demographic.columns
newcustomerlist_columns = NewCustomerList.columns
address_columns = Address.columns

In [107]:
# A code I prompted to generate a dataframe to generate a sheet vs column presence table
# This shows if a column header is present across multiple dataframes or not

def generate_presence_dataframe(*columns):
    """
    Generate a DataFrame to show the presence of attributes in each column.

    Args:
        *columns: Variable number of pandas DataFrame columns.

    Returns:
        A pandas DataFrame with the presence of attributes in each column.
        The column headers are based on the names of the passed columns,
        or generic names if the columns don't have names.
        The displayed columns are in the same order of the passed column arguments.
        The DataFrame is sorted based on the number of '1' values horizontally (across the rows).

    """
    # Step 1: Convert the column(s) to set(s)
    column_sets = [set(col) for col in columns]

    # Step 2: Create a set of all unique attributes from the column(s)
    all_attributes = sorted(list(set().union(*column_sets)))

    # Step 3: Create a dictionary to store the presence of attributes in each column
    presence_dict = {'Attributes': all_attributes}
    for i, col in enumerate(columns):
        column_name = col.name if col.name else f'Column {i+1}'
        presence_dict[column_name] = [1 if attr in col else 0 for attr in all_attributes]

    # Step 4: Create a DataFrame from the presence dictionary
    presence_df = pd.DataFrame(presence_dict)

    # Step 5: Sort the dataframe based on the number of '1' values horizontally (across the rows)
    presence_df = presence_df.iloc[presence_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=False).index]

    # Reset the index
    presence_df = presence_df.reset_index(drop=True)

    return presence_df


In [109]:
columns_presence_df = generate_presence_dataframe(transactions_columns, demographic_columns, address_columns,newcustomerlist_columns)
columns_presence_df

,Attributes,Column 1,Column 2,Column 3,Column 4
0,customer_id,1,1,1,0
1,DOB,0,1,0,1
2,first_name,0,1,0,1
3,tenure,0,1,0,1
4,state,0,0,1,1
5,property_valuation,0,0,1,1
6,postcode,0,0,1,1
7,past_3_years_bike_related_purchases,0,1,0,1
8,owns_car,0,1,0,1
9,job_title,0,1,0,1
